In [ ]:

!pip3 install ktrain

     |████████████████████████████████| 25.3MB 128kB/s 
     |████████████████████████████████| 6.8MB 52.4MB/s 
     |████████████████████████████████| 983kB 53.2MB/s 
     |████████████████████████████████| 266kB 56.8MB/s 
     |████████████████████████████████| 1.3MB 51.0MB/s 
     |████████████████████████████████| 1.2MB 51.5MB/s 
     |████████████████████████████████| 471kB 52.3MB/s 
     |████████████████████████████████| 2.9MB 22.7MB/s 
     |████████████████████████████████| 890kB 47.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=558d66779dbd6c65c00eaf32d7225250765b735922cbaaec1c04c1cb45c8af50
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=a22567d3766ad7df1be8231f46f558d6e27983802b3f6b4eaf2b739225dfbfa9
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

'0.25.4'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('text_emotion.csv')
data = data[['content','sentiment']]

data = data.dropna()
data = data.drop_duplicates()
data, data_test = train_test_split(data, test_size = 0.2, stratify = data[['sentiment']])
data.head()
data['sentiment'].value_counts()
#k=min(data['sentiment'].value_counts())

FileNotFoundError: ignored

In [ ]:
data.head()

,content,sentiment
24905,"@Fluffdoodle Lol, triplet. Haha I'm so glad t...",happiness
25158,"Sinus infection ): Damn brother. Anyways, Night",worry
19067,@dougiemcfly http://twitpic.com/67uc7 - can yo...,worry
32428,got to see 2 of my favorite guys back on long ...,love
10286,Goin to the mall and gonna see the movie ghost...,happiness


In [ ]:
data = data.groupby('sentiment',as_index = False,group_keys=False).apply(lambda s: s.sample(100,replace=True))
data['sentiment'].value_counts()
data.head()

,content,sentiment
19480,Sitting in traffic while my car gets rained on...,anger
22174,@paulmoxham everything is your fault moxy...ev...,anger
2629,Did a historical Jesus ever exist? Im finding ...,anger
2664,lost all my files since high school.,anger
5387,@mashable Link to ff &quot;how it works&quot; ...,anger


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data["content"].values, data.drop(['content'],axis=1).values, test_size=0.2, random_state=42)

In [ ]:

categories = ['anger','boredom','empty','enthusiasm','fun','happiness','hate','love','neutral','relief','sadness','surprise','worry']

print('size of training set: %s' % (len(x_train)))
print('size of validation set: %s' % (len(x_test)))
print('classes: %s' % (categories))


size of training set: 1040
size of validation set: 260
classes: ['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness', 'hate', 'love', 'neutral', 'relief', 'sadness', 'surprise', 'worry']


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

# model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
#MODEL_NAME = 'textattack/bert-base-uncased-yelp-polarity'
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=categories)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)


preprocessing train...
language: en
train sequence lengths:
	mean : 14
	95percentile : 25
	99percentile : 28


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 13
	95percentile : 24
	99percentile : 26


## STEP 2:  Load the BERT Model and Instantiate a Learner object

In [ ]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [ ]:
from keras.callbacks import ModelCheckpoint
filepath = "\content\saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=0,
    save_best_only=True, mode='auto', period=1)
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
174/174 [==============================] - 71s 364ms/step - loss: 2.5647 - accuracy: 0.0811 - val_loss: 2.5657 - val_accuracy: 0.0885
Epoch 2/4
174/174 [==============================] - 67s 385ms/step - loss: 2.5368 - accuracy: 0.1264 - val_loss: 2.4421 - val_accuracy: 0.1692
Epoch 3/4
174/174 [==============================] - 68s 390ms/step - loss: 2.2807 - accuracy: 0.3049 - val_loss: 2.2541 - val_accuracy: 0.2538
Epoch 4/4
174/174 [==============================] - 68s 392ms/step - loss: 1.8775 - accuracy: 0.4855 - val_loss: 2.1964 - val_accuracy: 0.2731


In [ ]:
# import te
print(checkpoint)
# new_model = tf.keras.models.load_model('content\saved-model-01')

#learner.validate(val_data=(x_test, y_test), class_names=categories)

In [ ]:
predictor = ktrain.get_predictor(learner.model, t)

In [ ]:
predictor.get_classes()

['anger',
 'boredom',
 'empty',
 'enthusiasm',
 'fun',
 'happiness',
 'hate',
 'love',
 'neutral',
 'relief',
 'sadness',
 'surprise',
 'worry']

In [ ]:
predictor.predict_proba("sam and sean are teasing me saying they are gonna get wings without me")

array([0.06207244, 0.0699559 , 0.04761993, 0.04359679, 0.04595064,
       0.04801569, 0.07620867, 0.05733221, 0.05403244, 0.10914754,
       0.18104683, 0.07169015, 0.13333075], dtype=float32)

In [ ]:
predictor.predict("sam and sean are teasing me saying they are gonna get wings without me")

'sadness'

In [ ]:
# we predicted the correct label
#print(test_b.target_names[test_b.target[0]])
# we predicted the correct label
print(data_test[0:6])

                                                 content sentiment
5084                              Not feeling very well.      love
16662  Drove a #Mazda #RX8 today and boy, that car is...   sadness
9777   @billbathgate hotttie poooh!!!! ooo i bet it s...       fun
16553  Then it just HAD to start raining here. ughh.....     worry
12793  too many meds. sleeepy sleeepy. I hate being sick     worry
16092                                 My cat is anorexic     worry


In [ ]:
# let's save the predictor for later use
predictor.save('/tmp/my_predictor')


In [ ]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor('/tmp/my_predictor')

In [ ]:
# make a prediction on the same document to verify it still works
reloaded_predictor.predict("*sigh* another two out fly out for Papi")

'sadness'